In [ ]:
import pandas as pd
import gspread

from typing import Any

json_file_path = "./data/drive-sheet-01.json"

In [ ]:
def googleSpreadData(url: str) -> Any:
    if url is None:
        return None

    gc = gspread.service_account(filename=json_file_path)
    # url = url

    ## 4. PythonSheet url을 바탕으로 스프레드 시트를 열고,
    doc = gc.open_by_url(url)

    workDf = pd.DataFrame()
    # print(doc.worksheets()[0].title)
    for sheet in doc.worksheets():
        print(sheet.title)
        data = doc.worksheet(sheet.title).get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])
        df["name"] = sheet.title
        workDf = pd.concat([workDf, df], ignore_index=True)
    # print(workDf)

    return workDf

In [ ]:
from langchain.tools import tool



@tool

def searchSheetData(query: str) -> str:
    """Search for things done by person's name"""


    url = "https://docs.google.com/spreadsheets/d/1rW-nTz7P69BHGCtp0amownjnuc2OJmFzJrQFoAfeKnc"

    dataDf = googleSpreadData(url)

    print(dataDf.head())

    return None

In [ ]:
searchSheetData.invoke({"query": ""})

### CSV-Excel-Agent 사용해보기

In [ ]:
# from langchain_experimental.tools import PythonAstREPLTool

# # 파이썬 코드를 실행하는 도구를 생성합니다.
# python_tool = PythonAstREPLTool()

url = "https://docs.google.com/spreadsheets/d/1rW-nTz7P69BHGCtp0amownjnuc2OJmFzJrQFoAfeKnc"
df = googleSpreadData(url)

In [12]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_teddynote.messages import AgentStreamParser, AgentCallbacks

agent = create_pandas_dataframe_agent(
    ChatOllama(
        base_url="http://172.26.81.72:11434",
        # model="gemma2:27b",
        model="llama3.1:latest",
        temperature=0.0,
        top_k=50,
        top_p=0.9,
        streaming=False,
    ),
    df,
    verbose=False,
    agent_type="tool-calling",
    allow_dangerous_code=True,
    prefix="You are a professional data analyst and expert in Pandas. "
    "You must use Pandas DataFrame(`df`) to answer user's request. "
    "\n\n[IMPORTANT] DO NOT create or overwrite the `df` variable in your code. \n\n"
    "If you are willing to generate visualization code, please use `plt.show()` at the end of your code. "
    "I prefer seaborn code for visualization, but you can use matplotlib as well.",
)

# parser_callback = AgentCallbacks(tool_callback, observation_callback, result_callback)
# stream_parser = AgentStreamParser(parser_callback)

In [ ]:
def ask(query):
    # 질의에 대한 답변을 출력합니다.
    response = agent.stream({"input": query})

    for step in response:
        # stream_parser.process_agent_steps(step)
        print(step)

In [ ]:
ask("몇 개의 행이 있어?")

In [ ]:
ask(
    "서울 여의도에서 시작하는 코스 이름들과 가장 긴 거리의 코스가 어디이고 몇 km인지 알려줘"
)

In [ ]:
ask(
    "거리가 가장 짧은 코스는 어디서 부터 어디까지 인지, 그리고 그 거리와 코스이름, 코스 설계자를 알려줘"
)

In [ ]:
ask("거리가 가장 짧은 코스들은 몇 km인지와 코스 이름들, 시작지점 종료지점을 알려줘.")

# SQLAgent 만들어 보기!!!

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
import psycopg2 as pg
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.agent_toolkits.sql.base import create_sql_agent

In [3]:
# DB접속 시 'postgresql+psycopg2://[계정]:[암호]@[서버:포트]/[db명]
db = SQLDatabase.from_uri("postgresql+psycopg2://ndschat:1@172.26.81.43:5432/ndsdb")
print(db)

In [48]:
llm = ChatOllama(
    base_url="http://172.26.81.72:11434",
    # model="gemma2:27b",
    model="llama3.1:latest",
    temperature=0.0,
    top_k=10,
    top_p=0.9,
    streaming=False,
)
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# prefix는 https://smith.langchain.com/hub/langchain-ai/sql-agent-system-prompt 가져옴.
# 반드시 dialect, top_k 가 포함되어야 함
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    prefix="You are an agent designed to interact with a SQL database.\n"
    "Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\n"
    "Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\n"
    "You can order the results by a relevant column to return the most interesting examples in the database.\n"
    "Never query for all the columns from a specific table, only ask for the relevant columns given the question.\n"
    "You have access to tools for interacting with the database.\n"
    "If you have single quotes in your SQL, make sure that the opening and closing quotes are paired.\n"
    "Be sure to add the 'public.' keyword to tables in the 'from' clause of SQL.\n"
    "Be sure to include the table alias in the columns of the SELECT statement. ex) u.id, c.id\n"
    "Each table's 'id' column is mapping others 'id' column.\n"
    "Even if you have questions in the results column, don't do any additional work to find answers to those questions.\n"
    "Only use the below tools. Only use the information returned by the below tools to construct your final answer.\n"
    "You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\n"
    "DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\n"
    "To start you should ALWAYS look at the tables in the database to see what you can query.\n"
    "Do NOT skip this step.\n"
    "Then you should query the schema of the most relevant tables.\n"
    "Please answer user question in Korean(Hangul)",
    top_k=5,
    # max_iterations=5,
)

In [50]:
agent_executor.invoke(
    "사용자명 '신재경'이 가장 마지막으로 등록한 1개의 인사와 관련된 질문이 뭐야?"
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input:chat_history_insa, chat_history_rule, userAction: sql_db_schema
Action Input: user, chat_history_insa
CREATE TABLE "user" (
	id VARCHAR NOT NULL, 
	name VARCHAR NOT NULL, 
	pw VARCHAR NOT NULL, 
	grp VARCHAR, 
	allow VARCHAR, 
	create_date TIMESTAMP WITHOUT TIME ZONE, 
	menu VARCHAR DEFAULT '1000000000'::character varying, 
	CONSTRAINT user_pk PRIMARY KEY (id)
)

/*
3 rows from user table:
id	name	pw	grp	allow	create_date	menu
0510190	신재경	1	DT개발팀	승인	None	1000000000
*/


CREATE TABLE chat_history_insa (
	id VARCHAR NOT NULL, 
	seq INTEGER NOT NULL, 
	chat_date VARCHAR NOT NULL, 
	question VARCHAR, 
	answer VARCHAR, 
	source_documents VARCHAR, 
	fill VARCHAR, 
	feedback VARCHAR, 
	create_date TIMESTAMP WITHOUT TIME ZONE, 
	update_date TIMESTAMP WITHOUT TIME ZONE, 
	CONSTRAINT chat_history_unique UNIQUE (id, seq)
)

/*
3 rows from chat_history_insa table:
id	seq	chat_date	question	answer	source_documents	

{'input': "사용자명 '신재경'이 가장 마지막으로 등록한 1개의 인사와 관련된 질문이 뭐야?",
 'output': '사용자명 \'신재경\'이 가장 마지막으로 등록한 1개의 인사와 관련된 질문은 "전배 신청 절차 좀 알려줘"입니다.'}